# Customer Review Sentiment Analysis

In [9]:
# pip install pandas nltk pyodbc sqlalchemy

import pandas as pd
import pyodbc 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import warnings 
warnings.filterwarnings("ignore")

In [10]:
# Download VADER Lexicon for sentiment analysis

nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\TOFAEL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
# Fetch data from SQL Server
def fetch_mssql_data():
    conn_str = (
        "DRIVER={SQL Server};"
        "SERVER=DESKTOP-TSJJHDQ\\SQLEXPRESS;"
        "DATABASE=marketing_data;"
        "Trusted_Connection=yes;"
    )
    
    conn = pyodbc.connect(conn_str)
    query = "SELECT * FROM customer_reviews"
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    return df

In [13]:
customer_reviews = fetch_mssql_data()
customer_reviews

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [14]:
# Initialize VADER sentiment intensity analyzer

sia = SentimentIntensityAnalyzer()

## Calculate Sentiment

In [17]:
# Calculate Sentiment Score using VADER

def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment["compound"]

In [24]:
customer_reviews["SentimentScore"] = customer_reviews["ReviewText"].apply(calculate_sentiment)
customer_reviews

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089
...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000


## Categorize Sentiment

In [28]:
# Categorize Sentiment using both sentiment score and review rating

def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return "Positive"
        elif rating == 3:
            return "Mixed Positive"
        else:
            return "Mixed Negative"
    elif score < -0.05:
        if rating <= 2:
            return "Negative"
        elif rating == 3:
            return "Mixed Negative"
        else:
            return "Mixed Positive"
    else:
        if rating >= 4:
            return "Positive"
        elif rating <= 2:
            return "Negative"
        else:
            return "Neutral"
    

In [33]:
customer_reviews["SentimentCategory"] = customer_reviews.apply(
    lambda row: categorize_sentiment(row["SentimentScore"], row["Rating"]), axis=1)

customer_reviews

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative
...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative


## Create Sentiment Bucket

In [34]:
# Bucket Sentiment Scores in to Text Ranges

def sentiment_bucket(score):
    if score >= 0.5:
        return "0.5 to 1.0"
    elif 0.0 <= score < 0.5:
        return "0.0 to 0.49"
    elif -0.5 <= 0.0:
        return "-0.49 to 0.0"
    else:
        return"-1.0 to -0.5"

In [36]:
customer_reviews["SentimentBucket"] = customer_reviews["SentimentScore"].apply(sentiment_bucket)
customer_reviews 

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative,-0.49 to 0.0
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative,-0.49 to 0.0
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive,0.5 to 1.0
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative,0.0 to 0.49


In [39]:
# Save the dataframe to csv with sentiment analysis

customer_reviews.to_csv("customer_review_sentiment.csv", index = False)


C:\Users\TOFAEL\customer_review_sentiment.csv


In [4]:
# Shows the full path where the file is saved

import os

file_path = os.path.abspath("customer_review_sentiment.csv")
print(file_path)  

C:\Users\TOFAEL\customer_review_sentiment.csv
